# Requisições e Tratamendo de dados

In [1]:
# importando bibliotecas 

import pandas as pd
import requests
import time
import ast

Obtendo cada marca de carro


In [ ]:
# trazendo as marcas de carros da API

url_marcas = 'https://parallelum.com.br/fipe/api/v2/cars/brands'
response_marcas = requests.get(url_marcas)
marcas = response_marcas.json()

In [ ]:
# tranformando em df e salvando como csv
marcas_df = pd.DataFrame(marcas)
marcas_df.to_csv('csv\marcas_fipe.csv',index=False)

Obtendo modelo de cada marca de carro

In [ ]:
# loop para consultar cada marca de carro e trazer os seus modelos

marcas_code = marcas_df['code'].tolist()
lista_modelos = []
for marca in marcas_code:
    url_modelos = f'https://parallelum.com.br/fipe/api/v2/cars/brands/{marca}/models'
    response_modelos = requests.get(url_modelos)
    modelos = response_modelos.json()
    modelos_df = pd.DataFrame(modelos)
    modelos_df['code_marcas'] = marca
    lista_modelos.append(modelos_df)

In [ ]:
# combinando os dataframes gerados de modelo e salvando em csv

modelos_fipe = pd.concat(lista_modelos, ignore_index=True)
modelos_fipe.rename(columns={'code':'code_modelo'},inplace=True)
modelos_fipe.to_csv('csv\modelos_fipe.csv',index=False)

Obtendo anos dos modelos de cada marca

In [ ]:
# loop para consultar cada marca e modelo de carro e trazer os seus anos

modelos_fipe_ = modelos_fipe
lista_anos_modelos = []

for index, row in modelos_fipe_.iterrows():
    c = row['code_marcas']
    v = row['code_modelo']
    
    url_anos_modelos = f'https://parallelum.com.br/fipe/api/v2/cars/brands/{c}/models/{v}/years'
    response_anos_modelos = requests.get(url_anos_modelos)
    
    if response_anos_modelos.status_code == 200:
        anos_modelos = response_anos_modelos.json()
        anos_modelos_df = pd.DataFrame(anos_modelos)
        anos_modelos_df['code_marca'] = c
        anos_modelos_df['code_modelos'] = v
        lista_anos_modelos.append(anos_modelos_df)
        print(f'Verificando {url_anos_modelos}')
        print(f'Marca {c}, modelo {v}')
    else:
        print(f'Erro na requisição para {url_anos_modelos}. Status code: {response_anos_modelos.status_code}')
    
    time.sleep(1)


In [ ]:
# combinando os dataframes gerados de cada ano de modelo e salvando em csv

lista_anos_modelos
anos_modelos_fipe = pd.concat(lista_anos_modelos, ignore_index=True)
anos_modelos_fipe.rename(columns={'code':'ano_modelo'},inplace=True)
anos_modelos_fipe.to_csv('csv\\anos_modelos_fipe.csv',index=False)

Obtendo o codigo fipe de cada modelo, marca e ano unico

Como cada modelo possui o mesmo codigo fipe independente do ano, queremos apenas um ano de cada marca e modelo

In [ ]:
# criando dataframe apenas com o primeiro ano de cada marca e modelo

anos_modelos_fipe.sort_values(by=['code_marca','code_modelos','ano_modelo'])
dados_para_codigo_fipe = anos_modelos_fipe.groupby(['code_marca', 'code_modelos'])['ano_modelo'].first().reset_index()

In [ ]:
# loop para consultar cada codigo fipe do modelo, marca e ano
lista_codigo_fipe = []

for index, row in dados_para_codigo_fipe.iterrows():
    m = row['code_marca']
    mod = row['code_modelos']
    y = row['ano_modelo']

    url_codigo_fipe = f'https://parallelum.com.br/fipe/api/v2/cars/brands/{m}/models/{mod}/years/{y}'

    response_codigo_fipe = requests.get(url_codigo_fipe)
    
    if response_codigo_fipe.status_code == 200:
        codigo_fipe = response_codigo_fipe.json()
        selecionando_valores = {
        'price': codigo_fipe['price'],
        'brand': codigo_fipe['brand'],
        'model': codigo_fipe['model'],
        'modelYear': codigo_fipe['modelYear'],
        'codeFipe': codigo_fipe['codeFipe']
        }
        codigo_fipe_df = pd.DataFrame([selecionando_valores])
        codigo_fipe_df['code_marca'] = m
        codigo_fipe_df['code_modelos'] = mod
        lista_codigo_fipe.append(codigo_fipe_df)
        print(f'Verificando {url_codigo_fipe}')
    else:
        print(f'Erro na requisição para {url_codigo_fipe}. Status code: {response_anos_modelos.status_code}')
    
    time.sleep(1)
print('Terminou a verificação')

In [ ]:
# combinando os dataframes gerados com cada codigo fipe para cada modelo e salvando csv

lista_codigo_fipe
modelos_codigo_fipe = pd.concat(lista_codigo_fipe, ignore_index=True)
modelos_codigo_fipe.head(30)
modelos_codigo_fipe.to_csv('csv\modelos_codigo_fipe.csv',index=False)

Criando chave de ligação (marca + modelo) para trazer o ano de cada codigo fipe

In [ ]:
# criando chave marca + modelo para join

modelos_codigo_fipe.head()
modelos_codigo_fipe['code_marca_modelo'] = modelos_codigo_fipe['code_marca'] + '_' + modelos_codigo_fipe['code_modelos']
anos_modelos_fipe.head()
anos_modelos_fipe['code_marca_modelo'] = anos_modelos_fipe['code_marca'] + '_' + anos_modelos_fipe['code_modelos']

In [ ]:
# realizando o join

df_code_fipe_ano_modelo = pd.merge(modelos_codigo_fipe[['code_marca_modelo','codeFipe']],
         anos_modelos_fipe[['code_marca_modelo','ano_modelo']],
         on='code_marca_modelo',how='inner')
df_code_fipe_ano_modelo.duplicated().sum()

In [ ]:
# removendo duplicatas e salvando em csv

df_code_fipe_ano_modelo_unique = df_code_fipe_ano_modelo.drop_duplicates(subset=['codeFipe','ano_modelo'])
df_code_fipe_ano_modelo
df_code_fipe_ano_modelo_unique.to_csv('csv\code_fipe_anos.csv',index=False)

Obtendo preço fipe para cada codigo fipe e ano do modelo

In [ ]:
# lendo arquvido csv e salvando em um dataframe

df = pd.read_csv('csv\code_fipe_anos.csv')

In [ ]:
# loop para cada codigo fipe e ano do modelo

lista_codigo_fipe_anos = []

timeout = 10

for index, row in df.iterrows():
    codeFipe = row['codeFipe']
    ano_modelo = row['ano_modelo']
    url_codigo_fipe_anos = f'https://parallelum.com.br/fipe/api/v2/cars/{codeFipe}/years/{ano_modelo}/history'

    try:
        response_codigo_fipe_anos = requests.get(url_codigo_fipe_anos, timeout=timeout)
        response_json = response_codigo_fipe_anos.json()

        if response_codigo_fipe_anos.status_code == 200:
            selecionando_valores = {
                'brand': response_json['brand'],
                'model': response_json['model'],
                'modelYear': response_json['modelYear'],
                'codeFipe': response_json['codeFipe'],
                'price_history': response_json['priceHistory']
            }
            selecionando_valores['ano_modelo'] = ano_modelo
            lista_codigo_fipe_anos.append(selecionando_valores)
            print(f'Conferindo {url_codigo_fipe_anos}')
        else:
            print(f'Erro na requisição para {url_codigo_fipe_anos}. Status code: {response_codigo_fipe_anos.status_code}')
    except requests.exceptions.RequestException as e:
        print(f'Erro na requisição para {url_codigo_fipe_anos}: {e}')

    time.sleep(5)
    

In [ ]:
# tranformar em dataframe e salvar em csv

if lista_codigo_fipe_anos:
    codigo_fipe_anos = pd.DataFrame(lista_codigo_fipe_anos)
    print('Processo concluído.')
else:
    print('Nenhum registro válido encontrado.')

codigo_fipe_anos.to_csv('csv\codigo_fipe_anos.csv',index=False)

Tratando json da coluna price_history

In [5]:
# ler csv e tranformar coluna price_history em objeto

df = pd.read_csv('csv\codigo_fipe_anos.csv')
df['price_history'] = df['price_history'].apply(ast.literal_eval)

In [6]:
# loop para cada linha da coluna price_history e expandindo os valores

df_expanded = []
for index, row in df.iterrows():
    for x in row['price_history']:
        df_expanded.append({
            'brand': row['brand'],
            'model': row['model'],
            'modelYear': row['modelYear'],
            'codeFipe': row['codeFipe'],
            'price': x['price'],
            'month': x['month'],
            'reference': x['reference'],
            'ano_modelo': row['ano_modelo']
        })

# criando novo dataframe com os valores expandidos
price_history = pd.DataFrame(df_expanded)

In [10]:
# salvando em csv

price_history.to_csv('csv\price_history.csv',index=False)